In [45]:
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import pystable

# Get Data

Coinbase data (treated)

In [23]:
coin_df = pd.read_csv('risk_pipeline/outputs/data/ETH-BTC_2022-06-01-00-00_2022-10-31-00-00_600secs_treated.csv', index_col=0)

In [24]:
coin_df.head()

,time,close
0,2022-06-01 00:00:00,0.061110
1,2022-06-01 00:10:00,0.061075
2,2022-06-01 00:20:00,0.060915
3,2022-06-01 00:30:00,0.060985
4,2022-06-01 00:40:00,0.061020


In [25]:
# invert close to match uni v3
coin_df.close = 1/coin_df.close

Uniswap data

In [26]:
uni_df = pd.read_csv('../csv/WBTC-WETH-10.0mTWAP-14881677-to-15864319.csv', index_col=0)

In [27]:
uni_df.head()

,timestamp,twap
0,2022-06-01 00:01:54,16.335589
2,2022-06-01 00:11:54,16.345683
4,2022-06-01 00:21:54,16.351692
10,2022-06-01 00:31:54,16.361773
11,2022-06-01 00:41:54,16.366600


In [28]:
uni_df.reset_index(inplace=True)

# Sanity checks

In [29]:
uni_df[uni_df.twap.isna()]

,index,timestamp,twap


In [30]:
coin_df[coin_df.close.isna()]

,time,close


In [31]:
print('shape:', uni_df.shape)
print('first timestamp:', uni_df.timestamp.min())
print('last timestamp:', uni_df.timestamp.max())

shape: (21883, 3)
first timestamp: 2022-06-01 00:01:54
last timestamp: 2022-10-30 23:01:54


In [32]:
print('shape:', coin_df.shape)
print('first timestamp:', coin_df.time.min())
print('last timestamp:', coin_df.time.max())

shape: (21889, 2)
first timestamp: 2022-06-01 00:00:00
last timestamp: 2022-10-31 00:00:00


# Combine data

In [33]:
df = pd.concat([coin_df.time, coin_df.close, uni_df.twap], axis=1)

In [34]:
df

,time,close,twap
0,2022-06-01 00:00:00,16.363934,16.335589
1,2022-06-01 00:10:00,16.373312,16.345683
2,2022-06-01 00:20:00,16.416318,16.351692
3,2022-06-01 00:30:00,16.397475,16.361773
4,2022-06-01 00:40:00,16.388069,16.366600
...,...,...,...
21884,2022-10-30 23:20:00,12.971010,NaN
21885,2022-10-30 23:30:00,12.965124,NaN
21886,2022-10-30 23:40:00,12.977743,NaN
21887,2022-10-30 23:50:00,12.965964,NaN


In [35]:
# Change column names
df.columns = ['time', 'coin', 'uni']

In [36]:
print('NAs in coinbase column =', df[df.coin.isna()].count()[0])
print('NAs in uniswap column =', df[df.uni.isna()].count()[0])


NAs in coinbase column = 0
NAs in uniswap column = 6


In [37]:
df.dropna(inplace=True)

In [38]:
df

,time,coin,uni
0,2022-06-01 00:00:00,16.363934,16.335589
1,2022-06-01 00:10:00,16.373312,16.345683
2,2022-06-01 00:20:00,16.416318,16.351692
3,2022-06-01 00:30:00,16.397475,16.361773
4,2022-06-01 00:40:00,16.388069,16.366600
...,...,...,...
21878,2022-10-30 22:20:00,13.005592,12.995307
21879,2022-10-30 22:30:00,12.997141,12.995306
21880,2022-10-30 22:40:00,12.990387,12.995304
21881,2022-10-30 22:50:00,12.971851,12.995304


# Analysis

In [40]:
def line_chart(df, title, xcol, ycol, x_title, y_title):
    fig = px.line(df, x=xcol, y=ycol)
    fig.update_layout(title=title)
    fig.update_layout(xaxis_title=x_title, yaxis_title=y_title)
    return fig

In [44]:
# Superimposed plot
plt = line_chart(df, 'Prices', 'time', ['coin', 'uni'], 'Time', 'Prices')
plt.show()

In [105]:
# Pystable funcs

def gaussian():
    return pystable.create(alpha=2.0, beta=0.0, mu=0.0,
                           sigma=1.0, parameterization=1)

def get_qs(dst):
    l = [i/1000 for i in range(10, 1000, 2)]
    return pystable.q(dst, l, len(l))

def pdf(dst, vals):
    return pystable.pdf(dst, vals, len(vals))

def log_returns(p):
    return [np.log(p[i]/p[i-1]) for i in range(1, len(p))]


In [131]:
log_coin = log_returns(df['coin'])
log_uni = log_returns(df['uni'])

In [133]:
len(coin_df)

21883

In [107]:
dst_coin = gaussian()
pystable.fit(dst_coin, log_coin, len(log_coin))

dst_uni = gaussian()
pystable.fit(dst_uni, log_uni, len(log_uni))

0

In [108]:
coin_q = get_qs(dst_coin)
uni_q = get_qs(dst_uni)

In [109]:
dist_df = pd.DataFrame(
    {'Percentage': [i/1000 for i in range(10, 1000, 2)],
     'coin': coin_q, 'uni': uni_q}
)

In [110]:
plt = line_chart(dist_df, 'CDFs', 'Percentage', ['coin', 'uni'], 'Percentage', 'Log returns')
plt.show()

In [135]:
df['uni_log_returns'] = [0] + log_uni
df['coin_log_returns'] = [0] + log_coin


In [140]:
fig = px.histogram(df[['uni_log_returns', 'coin_log_returns']], x=['uni_log_returns', 'coin_log_returns'])
fig.show()

In [139]:
df.sort_values('coin_log_returns', ascending=False)

,time,coin,uni,uni_log_returns,coin_log_returns
15351,2022-09-15 14:30:00,13.032712,12.722397,0.002834,0.021404
2064,2022-06-15 08:00:00,19.291984,18.949511,-0.000859,0.016645
7769,2022-07-24 22:50:00,14.104372,13.951985,-0.004304,0.015882
1882,2022-06-14 01:40:00,18.818216,18.581467,0.008509,0.014942
1529,2022-06-11 14:50:00,18.648019,18.574114,0.008089,0.014624
...,...,...,...,...,...
1783,2022-06-13 09:10:00,19.577134,20.042886,0.003393,-0.018873
14234,2022-09-07 20:20:00,11.752262,12.022531,-0.008783,-0.021681
21126,2022-10-25 17:00:00,13.579576,13.966913,-0.004797,-0.024260
2128,2022-06-15 18:40:00,18.426387,18.914845,-0.006385,-0.029356
